# Atom feed of tweeted new preprints by subject area

Preprints published in the past seven days tweeted at least three times as atom feed, grouped by subject area.

In [1]:
import sys
sys.path.append('../')
!{sys.executable} -m pip install lxml
import ast
import pandas as pd
from datetime import datetime, date
from feedgen.feed import FeedGenerator
import lxml

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [2]:
date = str(date.today().strftime('%Y-%m-%d'))
subject_areas = ['covid', 'medical_health-sciences', 'bioinformatics', 'biochemistry_cell-biology', 'cancer', 'ecology', 'evolutionary-biology', 'genetics', 'microbiology', 'neurosciences', 'physiology', 'plant-biology', 'zoology','other-biological-sciences']
preprints = pd.read_csv('../preprint_tweets/1101/preprint_tweets_' + date + '.csv', index_col=0)

# filter by section, and group all COVID-related preprints together
subsets = {}
subsets['covid'] = preprints[preprints['covid'] == True]
print('COVID: ' + str(len(subsets['covid'])))
subsets['medical_health-sciences'] = preprints[((preprints['archive'] == 'medRxiv')  | (preprints['subject-area'] == 'Pathology') | (preprints['subject-area'] == 'Pharmacology and Toxicology'))  & (preprints['covid'] == False)]
print('Medical and Health Sciences: ' + str(len(subsets['medical_health-sciences'])))
subsets['bioinformatics'] = preprints[(preprints['subject-area'] == 'Bioinformatics') & (preprints['covid'] == False)]
print('Bioinformatics: ' + str(len(subsets['bioinformatics'])))
subsets['biochemistry_cell-biology'] = preprints[((preprints['subject-area'] == 'Biochemistry') | (preprints['subject-area'] == 'Cell Biology') | (preprints['subject-area'] == 'Developmental Biology') | (preprints['subject-area'] == 'Synthetic Biology') | (preprints['subject-area'] == 'Systems Biology') | (preprints['subject-area'] == 'Molecular Biology')) & (preprints['covid'] == False)]
print('Biochemistry and Cell Biology: ' + str(len(subsets['biochemistry_cell-biology'])))
subsets['ecology'] = preprints[(preprints['subject-area'] == 'Ecology') & (preprints['covid'] == False)]
subsets['cancer'] = preprints[((preprints['subject-area'] == 'Cancer Biology') | (preprints['subject-area'] == 'Hematology') | (preprints['subject-area'] == 'Oncology')) & (preprints['covid'] == False)]
print('Cancer: ' + str(len(subsets['cancer'])))
print('Ecology: ' + str(len(subsets['ecology'])))
subsets['evolutionary-biology'] = preprints[(preprints['subject-area'] == 'Evolutionary Biology') & (preprints['covid'] == False)]
print('Evolutionary Biology: ' + str(len(subsets['evolutionary-biology'])))
subsets['genetics'] = preprints[(preprints['subject-area'] == 'Genetics') | (preprints['subject-area'] == 'Genomics') & (preprints['covid'] == False)]
print('Genetics: ' + str(len(subsets['genetics'])))
subsets['microbiology'] = preprints[((preprints['subject-area'] == 'Microbiology') | (preprints['subject-area'] == 'Immunology')) & (preprints['covid'] == False)]
print('Microbiology: ' + str(len(subsets['microbiology'])))
subsets['neurosciences'] = preprints[(preprints['subject-area'] == 'Neuroscience') & (preprints['covid'] == False)]
print('Neurosciences: ' + str(len(subsets['neurosciences'])))
subsets['physiology'] = preprints[(preprints['subject-area'] == 'Physiology') | (preprints['subject-area'] == 'Biophysics') & (preprints['covid'] == False)]
print('Physiology: ' + str(len(subsets['physiology'])))
subsets['plant-biology'] = preprints[(preprints['subject-area'] == 'Plant Biology') & (preprints['covid'] == False)]
print('Plant Biology: ' + str(len(subsets['plant-biology'])))
subsets['zoology'] = preprints[(preprints['subject-area'] == 'Zoology') | (preprints['subject-area'] == 'Animal Behavior and Cognition') & (preprints['covid'] == False)]
print('Zoology: ' + str(len(subsets['zoology'])))
subsets['other-biological-sciences'] = preprints[(preprints['archive'] != 'medRxiv') & (preprints['subject-area'] != 'Animal Behavior and Cognition') & (preprints['subject-area'] != 'Bioinformatics') & (preprints['subject-area'] != 'Biochemistry') & (preprints['subject-area'] != 'Biophysics') & (preprints['subject-area'] != 'Cancer Biology') & (preprints['subject-area'] != 'Cell Biology') & (preprints['subject-area'] != 'Developmental Biology') & (preprints['subject-area'] != 'Ecology') & (preprints['subject-area'] != 'Evolutionary Biology') & (preprints['subject-area'] != 'Genetics') & (preprints['subject-area'] != 'Genomics') & (preprints['subject-area'] != 'Immunology') & (preprints['subject-area'] != 'Microbiology') & (preprints['subject-area'] != 'Molecular Biology') & (preprints['subject-area'] != 'Neuroscience') & (preprints['subject-area'] != 'Physiology') & (preprints['subject-area'] != 'Plant Biology') & (preprints['subject-area'] != 'Synthetic Biology') & (preprints['subject-area'] != 'Systems Biology') & (preprints['subject-area'] != 'Zoology') & (preprints['covid'] == False)]
print('Other Biological Sciences: ' + str(len(subsets['other-biological-sciences'])))
subsets['microbiology'].head(50)

COVID: 35
Medical and Health Sciences: 19
Bioinformatics: 26
Biochemistry and Cell Biology: 46
Cancer: 7
Ecology: 14
Evolutionary Biology: 14
Genetics: 20
Microbiology: 46
Neurosciences: 35
Physiology: 14
Plant Biology: 5
Zoology: 3
Other Biological Sciences: 10


,doi,tweets,archive,subject-area,covid,title,authors,abstract,posted
11,10.1101/2022.01.13.476155,64,bioRxiv,Microbiology,False,Transatlantic spread of highly pathogenic avia...,"[{'name': 'V. Caliendo'}, {'name': 'N. S. Lewi...",<p>Highly pathogenic avian influenza (HPAI) vi...,2022-01-13
31,10.1101/2022.01.11.475790,32,bioRxiv,Microbiology,False,The phylogenomic landscape of the genus Serratia,"[{'name': 'David J. Williams'}, {'name': 'Patr...",<p>The genus <italic>Serratia</italic> has bee...,2022-01-11
43,10.1101/2022.01.11.475842,24,bioRxiv,Microbiology,False,A new class of protein sensor links spirochete...,"[{'name': 'A.R. Muok'}, {'name': 'K. Kurniyati...",<p>Pathogenic spirochetes can alter their morp...,2022-01-11
50,10.1101/2022.01.15.474735,19,bioRxiv,Microbiology,False,Comprehensive assessment of functional effects...,"[{'name': 'Zhongzhi Sun'}, {'name': 'Wenju Wan...",<p>The gut microbiome composition and function...,2022-01-16
52,10.1101/2022.01.13.476216,18,bioRxiv,Microbiology,False,The RNA polymerase gene specificity factor σ54...,"[{'name': 'Amy Switzer'}, {'name': 'Lynn Burch...",<p>The canonical function of a bacterial sigma...,2022-01-14
73,10.1101/2022.01.13.476242,14,bioRxiv,Microbiology,False,Local-scale virome depiction supports signific...,"[{'name': 'Arley Calle-Tobón'}, {'name': 'Juli...",<p><italic>Aedes</italic> spp. comprise the pr...,2022-01-13
74,10.1101/2022.01.14.476307,13,bioRxiv,Microbiology,False,Metabolic basis for the evolution of a common ...,"[{'name': 'Dallas L. Mould'}, {'name': 'Mirjan...",<p>Microbes frequently evolve in reproducible ...,2022-01-14
75,10.1101/2022.01.12.476089,13,bioRxiv,Immunology,False,Systematic analysis of human antibody response...,"[{'name': 'Elaine C. Chen'}, {'name': 'Pavlo G...",<p>Understanding the human antibody response t...,2022-01-13
85,10.1101/2022.01.13.476238,12,bioRxiv,Immunology,False,Bat Red Blood Cells express Nucleic Acid Sensi...,"[{'name': 'LK Metthew Lam'}, {'name': 'Jane Do...",<p>Red blood cells (RBCs) demonstrate immunomo...,2022-01-14
90,10.1101/2022.01.12.475240,12,bioRxiv,Microbiology,False,GR13-type plasmids in Acinetobacter potentiate...,"[{'name': 'Robert A. Moran'}, {'name': 'Haiyan...",<p>Carbapenem resistance and other antibiotic ...,2022-01-12


In [3]:

for subject_area in subject_areas:
    fg = FeedGenerator()
    fg.id('https://front-matter.io/')
    if subject_area == 'covid':
        fg.title(f'Popular BioRxiv/medRxiv COVID preprints posted in the last week')
    else:
        fg.title(f'Popular {subject_area.replace("-", " ").replace("_", " and ")} preprints posted in the last week')
    fg.author( {'name':'Martin Fenner','email':'martin@front-matter.io'} )
    fg.link( href='https://front-matter.io', rel='alternate' )
    fg.link( href=f'https://front-matter.io/{subject_area}/feed.xml', rel='self' )
    fg.language('en')

    for index, row in subsets[subject_area].iterrows():
        fe = fg.add_entry(order='append')
        fe.title(row['title'] + f" ({row['tweets']} tweets)")
        fe.author(ast.literal_eval(row['authors']))
        fe.id('https://doi.org/' + row['doi'])
        fe.link({'href': 'https://doi.org/' + row['doi'], 'title': row['title'] + f" ({row['tweets']} tweets)"})
        fe.description(row['abstract'])
        fe.category({ 'term': row['subject-area']})
        fe.published(datetime.fromisoformat(row['posted'] + 'T00:00:00+00:00'))
                 
    fg.atom_file(f'{subject_area}/feed.xml')